# Reinforcement Learning Agent for Statistical Arbitrage

This notebook demonstrates how to configure and train a reinforcement learning agent for statistical arbitrage using a snapshot model. The agent will learn to identify and exploit statistical arbitrage opportunities in financial markets.

## Install necessary libraries

The following libraries are required for this notebook. If you haven't installed them yet, you can do so using running the cell below or by using pip install in your terminal.

In [ ]:
%pip -q install -U numpy pandas pyarrow gdown gymnasium stable-baselines3 torch matplotlib tensorboard

## Configuration

The configuration section sets up the parameters for the reinforcement learning model.

In [1]:
CONFIG = {
    "DATA": {
        "drive_folder_id": "1uXEBUyySypdsW_ZqL-RZ3d1bWdIZisij", # google drive folder ID (can be found in the URL)
        "structure": {"1d":"ohlcv_1d", # subfolder structure inside the drive folder
                      "1h":"ohlcv_1h",
                      "15m":"ohlcv_15m",
                      "5m":"ohlcv_5m",
                      "1m":"ohlcv_1m"},
        "file_pattern": "{TICKER}_{FREQ}.parquet", # file naming pattern
        "tickers": ["BTC","ETH"], # pairs of assets to trade
        "sampling": "1h", # data sampling frequency (1m, 5m, 15m, 1h, 1d)
        "price_point": "close", # what price point to use for returns calculation (open, high, low, close, etc)
        "forward_fill": True, # forward fill missing data
        "drop_na_after_ffill": True, # drop rows with NA values after forward filling
        "cache_dir": "./data_cache", # local cache directory to store downloaded files
        "file_ids": {"BTC_1h": "1-sBNQpEFGEpVO3GDFCkSZiV3Iaqp2vB_", # map and define files to download (all data will be downloaded if left empty)
                     "ETH_1h": "1kj8G1scpFuEYTTXKEUzF9pwgGI2WFFL9"
                     }
    },
    "COINTEGRATION": {
        "drive_folder_id": "1V55Wx2MT-g6rQEo-ZEJIOSA-65zxLJYw", # google drive folder for cointegration files
        "file_pattern": "{ASSET1}_{ASSET2}_window_cointegration.csv", # cointegration file pattern
        "timestamp_format": "%Y-%m-%d %H:%M:%S", # format of timestamps in cointegration files
        "training_window_days": 2, # number of days to use for training after cointegration period
        "cache_dir": "./cointegration_cache" # cache directory for cointegration files
    },
    "ENV": {
        "include_cash": True, # include cash as a third asset
        "shorting": True, # allow shorting
        "lookback_window": 64, # lookback for feature slicing, meaning that the observation for each step will include data from the last 'lookback_window' time steps
        "features": {
            "vol_window": 64, # rolling volatility window
            "rsi_period": 14, # RSI lookback period
            "volume_change": True, # include volume change as a feature
            "normalize": True # normalize features using rolling z-score
        },
        "transaction_costs": { # transaction costs settings for exchange (e.g. Hyperliquid)
            "commission_bps": 5.0, # commission in basis points (bps)
            "slippage_bps": 5.0, # slippage in basis points (bps)
        },
        "reward": {
            "risk_lambda":0.001 # risk penalty coefficient (lambda), a.k.a. risk aversion factor
        },
        "leverage": {
            "long_cap": 2.0,  # maximum leverage for long positions
            "short_cap": 2.0, # maximum leverage for short positions
            "use_asymmetric": True, # whether to allow different caps for long and short positions
        },
        "constraints": {
            "min_weight": -2.0, # minimum weight for each asset (for shorting)
            "max_weight": 2.0,  # maximum weight for each asset (for leverage)
            "sum_to_one": False # whether weights must sum to one (need to be False for leverage)
        },
        "seed": 42 # random seed for reproducibility
    },
    "SPLITS": { # date splits for training, validation, and testing
        "data_start":"2024-09-02", # start date of the entire dataset
        "data_end":"2025-09-02", # end date of the entire dataset
        "train":["2024-09-02","2025-06-15"], # training period
        "val":["2025-06-16","2025-07-15"], # validation period
        "test":["2025-07-16","2025-09-02"], # testing period
        "walk_forward": True, # whether to use walk-forward splits (e.g. sliding window)
        "wf_train_span_days": 180, # training window span in days for walk-forward
        "wf_test_span_days": 30, # testing window span in days for walk-forward
        "wf_step_days": 30 # step size in days to move the window forward
    },
    "RL": {
        "timesteps":10,
        "policy":"MlpPolicy",
        "gamma":0.99,
        "gae_lambda":0.95,
        "clip_range":0.2,
        "n_steps":1024,
        "batch_size":256,
        "learning_rate":3e-4,
        "ent_coef":0.0,
        "vf_coef":0.5,
        "max_grad_norm":0.5
    },
    "EVAL": {
        "plots":True,
        "reports_dir":"./reports"
    },
    "IO": {
        "models_dir":"./models",
        "tb_logdir":"./tb"
    }
}

## Imports

In [ ]:
import os
import json
import gdown
import math
import glob
import random
import pytz

import time

from datetime import timedelta, datetime

from typing import Dict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reinforcement Learning
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor

# Deep Learning
import torch

## Set Computation Device

This section sets the computation device for training the model. It checks if a GPU is available and sets it as the device; otherwise, it defaults to CPU.

In [3]:
# run on cuda GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")

# run on Apple Silicon
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS is available. Using Apple Silicon GPU.")

# run on CPU (slow)
else:
    device = torch.device("cpu")
    print("CUDA and MPS are not available. Using CPU.")

MPS is available. Using Apple Silicon GPU.


## Set seeds for reproducibility

This section sets the random seeds for various libraries to ensure that the results are reproducible.

Note: It is good practice to type (set data types) for function and method parameters for better code maintainability.

In [4]:
def set_all_seeds(seed: int = 42):
    random.seed(seed) # seed for random module
    np.random.seed(seed) # seed for numpy module
    try:
        torch.manual_seed(seed) # seed for torch module
        if torch.cuda.is_available(): # seed for CUDA device
            torch.cuda.manual_seed_all(seed)
        elif torch.backends.mps.is_available(): # seed for Apple Silicon device
            torch.backends.mps.manual_seed_all(seed)
    except Exception:
        pass

set_all_seeds(CONFIG["ENV"]["seed"]) # set all seeds for reproducibility

# set annualization factors for different timeframes
ANNUALIZATION = {"1m":365*24*60,
                 "5m":365*24*12,
                 "15m":365*24*4,
                 "1h":365*24,
                 "1d":365}

## Fetch Data from Google Drive

This section handles downloading data from Google Drive. It supports two methods: downloading an entire folder or downloading specific files by their IDs. The data will be cached locally for faster subsequent loading.

In [7]:
ROOT_ID = CONFIG["DATA"]["drive_folder_id"] # suffix for Google Drive download URL
CACHE_DIR = CONFIG["DATA"]["cache_dir"] # data will be cached here for faster subsequent loading
FILE_IDS = CONFIG["DATA"]["file_ids"] # mapping of data to its URL suffix

download_all = True if FILE_IDS == {} else False # this determines whether to download entire folder or specific files by their IDs (s. CONFIG)

# create directory if it doesn't already exist
def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

# download entire folder from Google Drive with retry logic
def download_drive_folder(root_id: str, out_dir: str, max_retries: int = 3):
    print("Mirroring Google Drive folder locally...")
    
    for attempt in range(max_retries):
        try:
            gdown.download_folder(
                id=root_id, 
                output=out_dir, 
                quiet=False, 
                use_cookies=False,
                remaining_ok=True  # Continue even if some files fail
            )
            print("Folder mirroring complete.")
            return True
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Download attempt {attempt + 1} failed. Retrying... Error: {str(e)}")
                time.sleep(5)  # Wait 5 seconds before retrying
            else:
                print(f"All download attempts failed for folder {root_id}. Error: {str(e)}")
                return False

# download specific files from Google Drive by their file IDs with retry logic
def targeted_download_by_ids(file_id_map: Dict[str, str], out_dir: str, max_retries: int = 3):
    ensure_dir(out_dir)
    
    for name, fid in file_id_map.items():
        # check if file already exists in local cache folder
        if os.path.exists(os.path.join(out_dir, name)) or os.path.exists(os.path.join(out_dir, f"{name}.parquet")):
            print(f"File {name} already exists in cache. Skipping download.")
            continue
        
        suffix = name if name.endswith(".parquet") else f"{name}.parquet" # ensure .parquet suffix
        out_path = os.path.join(out_dir, suffix) # full output path
        
        for attempt in range(max_retries):
            try:
                print(f"Downloading {name} -> {out_path}")
                url = f"https://drive.google.com/uc?id={fid}"
                success = gdown.download(url, out_path, quiet=False, use_cookies=False, verify=False)
                if success:
                    break
                else:
                    raise Exception("Download failed")
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"Download attempt {attempt + 1} failed for {name}. Retrying... Error: {str(e)}")
                    time.sleep(5)  # Wait 5 seconds before retrying
                else:
                    print(f"All download attempts failed for {name}. Error: {str(e)}")

# check if cache directory exists, if not create it
ensure_dir(CACHE_DIR)

if download_all: # download entire folder
    download_drive_folder(ROOT_ID, CACHE_DIR)
else: # download specific files by their IDs
    targeted_download_by_ids(FILE_IDS, CACHE_DIR)

print("Download step complete.")

File BTC_1h already exists in cache. Skipping download.
File ETH_1h already exists in cache. Skipping download.
Download step complete.


## Load data

Once the data is downloaded, this section loads the data into a pandas DataFrame for further processing.

## Load Cointegration Data

This section loads the cointegration test results from CSV files and processes them to identify valid training periods.

In [ ]:
# Download cointegration data
COINT_ROOT_ID = CONFIG["COINTEGRATION"]["drive_folder_id"]
COINT_CACHE_DIR = CONFIG["COINTEGRATION"]["cache_dir"]
ensure_dir(COINT_CACHE_DIR)

# Download cointegration folder with improved error handling
print("Downloading cointegration data...")
success = download_drive_folder(COINT_ROOT_ID, COINT_CACHE_DIR)

if not success:
    print("Warning: Some cointegration files may be missing. Continuing with available data.")

def get_all_assets_from_cointegration():
    """Scan cointegration files to get all unique assets."""
    assets = set()
    pattern = os.path.join(COINT_CACHE_DIR, "**", "*_window_cointegration.csv")
    coint_files = glob.glob(pattern, recursive=True)
    
    for file_path in coint_files:
        base_name = os.path.basename(file_path)
        pair_assets = base_name.replace("_window_cointegration.csv", "").split("_")[:2]
        assets.update(pair_assets)
    
    return sorted(list(assets))

def load_cointegration_data():
    """Load all cointegration test results and return a DataFrame with training windows."""
    all_periods = []
    
    # Find all cointegration CSV files
    pattern = os.path.join(COINT_CACHE_DIR, "**", "*_window_cointegration.csv")
    coint_files = glob.glob(pattern, recursive=True)
    
    if not coint_files:
        print("Warning: No cointegration files found in cache directory.")
        return pd.DataFrame()
    
    for file_path in coint_files:
        try:
            # Extract asset names from filename
            base_name = os.path.basename(file_path)
            assets = base_name.replace("_window_cointegration.csv", "").split("_")[:2]
            
            # Read cointegration results
            df = pd.read_csv(file_path)
            df['start'] = pd.to_datetime(df['start'], format=CONFIG["COINTEGRATION"]["timestamp_format"])
            df['end'] = pd.to_datetime(df['end'], format=CONFIG["COINTEGRATION"]["timestamp_format"])
            
            # Filter for cointegrated periods
            coint_periods = df[df['cointegrated']]
            
            for _, row in coint_periods.iterrows():
                # Calculate training period (2 days after cointegration period)
                train_start = row['end']
                train_end = train_start + pd.Timedelta(days=CONFIG["COINTEGRATION"]["training_window_days"])
                
                all_periods.append({
                    'asset1': assets[0],
                    'asset2': assets[1],
                    'coint_start': row['start'],
                    'coint_end': row['end'],
                    'train_start': train_start,
                    'train_end': train_end,
                    'alpha': row['alpha'],
                    'beta': row['beta'],
                    'adf_p': row['adf_p'],
                    'correlation': row['correlation']
                })
        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")
            continue
    
    return pd.DataFrame(all_periods)

# Get all unique assets from cointegration files
all_assets = get_all_assets_from_cointegration()
print(f"Found {len(all_assets)} unique assets in cointegration files: {all_assets}")

# Update CONFIG with discovered assets
CONFIG["DATA"]["tickers"] = all_assets

# Load cointegration data
coint_df = load_cointegration_data()
print(f"Loaded {len(coint_df)} cointegration periods")

Mirroring Google Drive folder locally...


Retrieving folder contents


Processing file 1YZJXQ2fZNyP5xO9U2NTkyBnEexLB9odN ADA_DOGE_window_cointegration.csv
Processing file 136YCiC1IscZpZn6hoOf3US7tGet4p1Aj ADA_HBAR_window_cointegration.csv
Processing file 18iktTkGXlTNkCaZ_-C3kAVzPTjQv-quE ADA_LINK_window_cointegration.csv
Processing file 1H2b8ZIBK5jjFUgQ8KT0yPBfsQoRRuF8p ADA_LTC_window_cointegration.csv
Processing file 1UGyabqR_vBbVZ0PcSBmGy8dftKdMQBVt ADA_SUI_window_cointegration.csv
Processing file 1DZtQzzXeIkxlCNc4_QLuIDXeth3cGpxi ADA_XLM_window_cointegration.csv
Processing file 11urUO0yJzjDxPYAd0YpYiCugHlQA70Ft ADA_XRP_window_cointegration.csv
Processing file 1OurOhnonlKXtpDSi1keAuARoeXy9vEXw APT_ARB_window_cointegration.csv
Processing file 1EHHIl_07HpbzMYkYMMeVzl9R5xBR-usN APT_AVAX_window_cointegration.csv
Processing file 1BWe7E8-IPROlxsuQCqxsgisQv0O5zIhF APT_NEAR_window_cointegration.csv
Processing file 1JMjHuSVv7sHzi5cbgZHbkRL_0gKOlnQ- APT_TON_window_cointegration.csv
Processing file 1def47NyM3EZJHavN1ip7NrF2JVxTDPJs APT_WLD_window_cointegration.csv

Retrieving folder contents completed
Building directory structure
Building directory structure completed


Download attempt 1 failed. Retrying... Error: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1YZJXQ2fZNyP5xO9U2NTkyBnEexLB9odN

but Gdown can't. Please check connections and permissions.


NameError: name 'time' is not defined

In [9]:
sampling = CONFIG["DATA"]["sampling"] # data sampling frequency (1m, 5m, 15m, 1h, 1d)
subfolder = CONFIG["DATA"]["structure"][sampling] # subfolder name inside the drive folder is determined by the sampling frequency
pattern_fmt = CONFIG["DATA"]["file_pattern"] # file naming pattern (s. CONFIG)
tickers = CONFIG["DATA"]["tickers"] # list of tickers to load
forward_fill = CONFIG["DATA"]["forward_fill"] # whether to forward fill missing data
drop_na_after_ffill = CONFIG["DATA"]["drop_na_after_ffill"] # whether to drop NA values after forward filling

# function to find parquet file path
def find_parquet_path(ticker: str, sampling: str) -> str:
    fname = pattern_fmt.format(TICKER=ticker, FREQ=sampling)
    # try subfolder first
    candidates = glob.glob(os.path.join(CACHE_DIR, "**", subfolder, fname), recursive=True)
    # if not found, try flat cache
    if not candidates:
        candidates = glob.glob(os.path.join(CACHE_DIR, "**", fname), recursive=True)
    # if still not found, try direct file in cache (for file_ids downloads)
    if not candidates:
        direct_path = os.path.join(CACHE_DIR, fname)
        if os.path.exists(direct_path):
            candidates = [direct_path]
    # if nothing is found at all, raise an error
    if not candidates:
        raise FileNotFoundError(f"Could not find {fname} under {CACHE_DIR}.")
    return candidates[0]

# function to localize timestamps and align dataframes
def localize_and_align(df: pd.DataFrame, tz_in: str = None, tz_out: str = None) -> pd.DataFrame:
    # convert millisecond timestamps to datetime
    if 'datetime' in df.columns:
        df['timestamp'] = pd.to_datetime(df['datetime'], unit='ms', utc=True)
        df = df.set_index('timestamp')
    # make column names lowercase for consistency
    cols = {c: c.lower() for c in df.columns}
    df = df.rename(columns=cols)
    return df.sort_index()

dfs = {}
for t in tickers:
    pth = find_parquet_path(t, sampling) # find file path
    tmp = pd.read_parquet(pth) # read parquet file
    tmp = localize_and_align(tmp) # standardize timestamps and column names
    if forward_fill: # forward fill missing data if specified
        tmp = tmp.ffill()
    if drop_na_after_ffill: # drop rows with NA values after forward filling
        tmp = tmp.dropna()
    dfs[t] = tmp # store in dictionary

common_index = None
# find common index across all dataframes
for t, df in dfs.items():
    common_index = df.index if common_index is None else common_index.intersection(df.index)
# reindex all dataframes to the common index and drop any remaining NA values
for t in tickers:
    dfs[t] = dfs[t].reindex(common_index).dropna()

# print the shape of each dataframe to ensure alignment
print({t: dfs[t].shape for t in tickers})

{'BTC': (8785, 5), 'ETH': (8785, 5)}


## Feature engineering

This section performs feature engineering on the loaded data. It includes creating new features, normalizing data, and preparing the dataset for training the reinforcement learning agent. To ensure statistical arbitrage strategy is imitated, create relevant features.

In [10]:
feat_cfg = CONFIG["ENV"]["features"]
price_col = CONFIG["DATA"]["price_point"]

# function to compute relative strength index (momentum indicator)
def compute_rsi(series: pd.Series, period: int = 14) -> pd.Series:
    delta = series.diff() # compute price changes
    up = (delta.clip(lower=0)).ewm(alpha=1/period, adjust=False).mean() # average of upward price changes
    down = (-delta.clip(upper=0)).ewm(alpha=1/period, adjust=False).mean() # average of downward price changes
    rs = up / (down + 1e-12) # relative strength ratio indicates if upward or downward momentum is stronger
    rsi = 100 - (100 / (1 + rs)) # converts RS to RSI bounded between 0 and 100
    return rsi

# function to create features dataframe
def make_features(df: pd.DataFrame, price_col: str, vol_window: int, rsi_period: int, volume_change: bool):
    out = pd.DataFrame(index=df.index) # initialize output dataframe
    out["ret"] = np.log(df[price_col]).diff(1) # log returns
    out["vol"] = out["ret"].rolling(vol_window).std().fillna(0.0) # rolling volatility
    out["rsi"] = compute_rsi(df[price_col], rsi_period).fillna(50.0) # relative strength index
    if "volume" in df.columns and volume_change: # log volume change if volume data is available
        out["volchg"] = np.log(df["volume"].replace(0, np.nan)).diff().fillna(0.0) # log volume change
    else:
        out["volchg"] = 0.0 # if no volume data, set to zero
    return out

features_by_ticker = {}
for t in tickers: # create features for each ticker
    fdf = make_features(dfs[t], price_col, feat_cfg["vol_window"], feat_cfg["rsi_period"], feat_cfg["volume_change"])
    features_by_ticker[t] = fdf # store features in dictionary

panel_cols = []
for t in tickers: # create multi-index columns
    for col in ["ret","vol","rsi","volchg"]:
        panel_cols.append((t, col))
panel = pd.concat([features_by_ticker[t][["ret","vol","rsi","volchg"]] for t in tickers], axis=1) # combine features horizontally
panel.columns = pd.MultiIndex.from_tuples(panel_cols, names=["ticker","feature"]) # set multi-index columns
panel = panel.dropna() # remove rows with missing values

print(panel.tail())
panel.describe()

ticker                    BTC                                      ETH  \
feature                   ret       vol        rsi    volchg       ret   
datetime                                                                 
2025-09-02 18:00:00 -0.002415  0.004553  56.046963 -0.326068 -0.005253   
2025-09-02 19:00:00  0.001441  0.004556  57.347919 -0.123407 -0.005208   
2025-09-02 20:00:00  0.005512  0.004575  61.998583  0.267870  0.009015   
2025-09-02 21:00:00 -0.002596  0.004589  58.744666 -0.496680  0.003603   
2025-09-02 22:00:00  0.000574  0.004588  59.252994  0.016223 -0.000814   

ticker                                              
feature                   vol        rsi    volchg  
datetime                                            
2025-09-02 18:00:00  0.006955  43.078511  0.493765  
2025-09-02 19:00:00  0.006879  40.698143 -0.197759  
2025-09-02 20:00:00  0.006964  46.245797 -0.063885  
2025-09-02 21:00:00  0.006982  48.339198 -0.467361  
2025-09-02 22:00:00  0.006952  47.88

ticker           BTC                                                 ETH  \
feature          ret          vol          rsi       volchg          ret   
count    8784.000000  8784.000000  8784.000000  8784.000000  8784.000000   
mean        0.000075     0.004585    51.232259    -0.000192     0.000067   
std         0.004991     0.001950    11.619616     0.528632     0.007677   
min        -0.050195     0.000000     0.000000    -2.149282    -0.124892   
25%        -0.002048     0.003331    43.736511    -0.347504    -0.003253   
50%         0.000064     0.004109    51.268990    -0.024810     0.000131   
75%         0.002256     0.005470    58.885158     0.296147     0.003631   
max         0.049047     0.013790    90.153680     2.690468     0.091261   

ticker                                          
feature          vol          rsi       volchg  
count    8784.000000  8784.000000  8784.000000  
mean        0.007127    51.206699    -0.000106  
std         0.002781    12.326999     0.546888  
min         0.000000     6.828906    -2.068495  
25%         0.005369    43.299516    -0.360170  
50%         0.006657    51.018342    -0.043595  
75%         0.008315    58.761127     0.317103  
max         0.022406   100.000000     2.696904

## Feature scaling and state tensor construction

This section normalizes the features and constructs the state tensors required for training the reinforcement learning agent. State tensors are multi-dimensional arrays that represent the current state of the environment that the RL agent uses to make decisions.

In [11]:
lookback = CONFIG["ENV"]["lookback_window"] # lookback window 
normalize = CONFIG["ENV"]["features"]["normalize"] # normalization method

# rolling z-score normalization for features
def rolling_zscore(df: pd.DataFrame, window: int = 256) -> pd.DataFrame:
    mu = df.rolling(window).mean() # rolling mean
    sigma = df.rolling(window).std().replace(0, np.nan) # rolling std dev, replace 0 with NaN to avoid division by zero
    z = (df - mu) / (sigma + 1e-12) # z-score normalization
    return z.fillna(0.0) # fill NaNs with 0.0 and return z-scored dataframe

# build state tensor
def build_state_tensor(panel: pd.DataFrame, lookback: int, normalize: bool = False):
    # normalization step for each feature
    if normalize: # group by feature and apply z-score while preserving MultiIndex
        normalized = pd.DataFrame(index=panel.index) # empty dataframe with same index as features dataframes to hold scaled features
        for feature in panel.columns.unique(level=1): # iterate over features
            feature_data = panel.xs(feature, level=1, axis=1)
            z_scored = rolling_zscore(feature_data, window=max(lookback*2, 256)) # take double the lookback value for rolling
            for ticker in z_scored.columns: # reconstruct MultiIndex columns
                normalized[(ticker, feature)] = z_scored[ticker]
        normalized.columns = pd.MultiIndex.from_tuples(normalized.columns, names=["ticker", "feature"]) # set MultiIndex columns
    else: # no normalization, use raw features
        normalized = panel.copy()

    # organize data by tickers, features, and time
    tickers = sorted(panel.columns.unique(level=0)) # ensure consistent order
    features = sorted(panel.columns.unique(level=1)) # feature order
    times = normalized.index # time index

    # create sliding windows of lookback length
    # extract feature data for the window and stack into a 3D tensor (tickers x features x lookback)
    X, y_ret, inst_vol = [], [], [] # lists to hold state tensors, next returns, and current volatilities
    for i in range(lookback, len(times)-1): # iterate over time index in lookback window to have a sliding window with each time point as a step
        window_slice = normalized.iloc[i-lookback:i]
        frames = []
        for t in tickers: # save the windows of each ticker
            frames.append(window_slice[t].T.values)
        tensor = np.stack(frames, axis=0) # stack into 3D tensor
        X.append(tensor) # append tensor to list
        nxt = panel.iloc[i+1] # next period returns
        y_ret.append(np.array([nxt[(t, "ret")] for t in tickers], dtype=float)) # save to list
        cur = panel.iloc[i] # current period volatilities
        inst_vol.append(np.array([cur[(t, "vol")] for t in tickers], dtype=float)) # save to list

    X = np.array(X, dtype=np.float32) # convert list of state tensors of tensors to 3D numpy array
    y_ret = np.array(y_ret, dtype=np.float32) # convert list of next returns to 2D numpy array
    inst_vol = np.array(inst_vol, dtype=np.float32) # convert list of volatilities to 2D numpy array
    return X, y_ret, inst_vol, tickers, features, times[lookback+1:]

X_all, R_all, VOL_all, TICKER_ORDER, FEAT_ORDER, TIME_INDEX = build_state_tensor(
    panel, lookback=lookback, normalize=normalize
)

print("State tensor:", X_all.shape, "Returns:", R_all.shape, "InstVol:", VOL_all.shape)

State tensor: (8719, 2, 4, 64) Returns: (8719, 2) InstVol: (8719, 2)


## Define Splits and Adjust Timezones

This section defines the training and validation splits for the dataset. It ensures that the data is divided appropriately to train the model, validate its performance during training, and test its final performance on unseen data. Also, it adjusts the timezones of the datetime indices to ensure consistency across the dataset. This is necessary in case the data comes from multiple sources with different timezone settings.

In [ ]:
# function to create boolean mask for date slicing
def date_slice_mask(times: pd.DatetimeIndex, start: str, end: str):
    # convert input dates to UTC timestamps
    start_ts = pd.Timestamp(start).tz_localize('UTC')
    end_ts = pd.Timestamp(end).tz_localize('UTC')
    
    # ensure time index is in UTC
    if times.tz is None: # if naive, localize to UTC
        times = times.tz_localize('UTC')
    elif times.tz != pytz.UTC: # if timezone-aware but not UTC, convert to UTC
        times = times.tz_convert('UTC')
        
    return (times >= start_ts) & (times <= end_ts) # return boolean mask for splits later on

def create_cointegration_splits(coint_df: pd.DataFrame, price_data_index: pd.DatetimeIndex):
    """Create training windows based on cointegration periods."""
    train_masks = []
    val_masks = []
    test_masks = []
    
    # Ensure indices are timezone-aware
    if price_data_index.tz is None:
        price_data_index = price_data_index.tz_localize('UTC')
    
    # Sort periods chronologically
    coint_df = coint_df.sort_values('train_start')
    
    # Split cointegration periods into train/val/test
    split_point1 = int(0.7 * len(coint_df))  # 70% for training
    split_point2 = int(0.85 * len(coint_df))  # 15% for validation
    
    train_periods = coint_df.iloc[:split_point1]
    val_periods = coint_df.iloc[split_point1:split_point2]
    test_periods = coint_df.iloc[split_point2:]
    
    # Create masks for each period type
    for periods, masks in [(train_periods, train_masks), 
                          (val_periods, val_masks), 
                          (test_periods, test_masks)]:
        for _, row in periods.iterrows():
            mask = (price_data_index >= row['train_start']) & (price_data_index <= row['train_end'])
            masks.append(mask)
    
    # Combine masks for each split with OR operation
    train_mask = np.logical_or.reduce(train_masks) if train_masks else np.zeros(len(price_data_index), dtype=bool)
    val_mask = np.logical_or.reduce(val_masks) if val_masks else np.zeros(len(price_data_index), dtype=bool)
    test_mask = np.logical_or.reduce(test_masks) if test_masks else np.zeros(len(price_data_index), dtype=bool)
    
    print(f"Training windows: {train_mask.sum()} timesteps")
    print(f"Validation windows: {val_mask.sum()} timesteps")
    print(f"Testing windows: {test_mask.sum()} timesteps")
    
    return train_mask, val_mask, test_mask

# Create training masks based on cointegration periods
train_mask, val_mask, test_mask = create_cointegration_splits(coint_df, TIME_INDEX)

# Create splits for training
SPLITS = [{
    "name": "CointegrationSplit",
    "train": train_mask,
    "val": val_mask,
    "test": test_mask
}]

NameError: name 'coint_df' is not defined

## Environment Configuration

This is a custom Gymnasium environment for portfolio optimization.

In [ ]:
class PortfolioWeightsEnv(gym.Env):
    metadata = {"render_modes": []}

    def __init__(self, X, R, VOL, tickers, lookback, cfg_env):
        super().__init__()
        self.X = X # state tensor
        self.R = R # next period returns
        self.VOL = VOL # current period volatilities
        self.tickers = tickers # list of assets
        self.lookback = lookback # lookback window
        self.cfg = cfg_env # environment configuration taken from CONFIG

        self.n_assets = len(tickers) # number of assets
        self.include_cash = cfg_env["include_cash"] # whether to include cash as an asset
        self.shorting = cfg_env["shorting"] # whether shorting is allowed
        self.dim_action = self.n_assets + (1 if self.include_cash else 0) # action space dimension: one weight per asset (+1 for cash if included)

        obs_dim = self.n_assets * self.X.shape[2] * self.lookback # observation space dimension: tickers x features x lookback
        self.observation_space = spaces.Box(low=-5, high=5, shape=(obs_dim,), dtype=np.float32) # needed for Gym to define feature bounds (±5 for z-scored features)
        
        # Update action space to allow for leveraged positions
        if self.shorting:
            self.action_space = spaces.Box(
                low=-cfg_env["leverage"]["short_cap"],
                high=cfg_env["leverage"]["long_cap"],
                shape=(self.dim_action,),
                dtype=np.float32
            )
        else:
            self.action_space = spaces.Box(
                low=0.0,
                high=cfg_env["leverage"]["long_cap"],
                shape=(self.dim_action,),
                dtype=np.float32
            )

        self.commission = cfg_env["transaction_costs"]["commission_bps"] / 1e4 # commission in decimal form
        self.slippage = cfg_env["transaction_costs"]["slippage_bps"] / 1e4 # slippage in decimal form
        self.risk_lambda = cfg_env["reward"]["risk_lambda"] # defines risk aversion in reward function

        # Leverage settings
        self.long_cap = cfg_env["leverage"]["long_cap"]
        self.short_cap = cfg_env["leverage"]["short_cap"]
        self.use_asymmetric = cfg_env["leverage"]["use_asymmetric"]

        self.reset(seed=cfg_env.get("seed", 42))

    def _to_obs(self, t):
        # Get the current window of observations
        arr = self.X[t].reshape(-1).astype(np.float32)
        return arr
    
    # project raw actions to asset weights with leverage
    def _project_weights(self, a):
        if self.use_asymmetric:
            # Apply asymmetric leverage caps
            w = np.clip(a, -self.short_cap if self.shorting else 0.0, self.long_cap)
        else:
            # Use symmetric leverage cap
            max_leverage = max(self.long_cap, self.short_cap)
            w = np.clip(a, -max_leverage if self.shorting else 0.0, max_leverage)
        return w

    # reset environment to initial state
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.t = 0 # reset time index
        self.portfolio_value = 1.0 # reset portfolio value
        self.w = np.zeros(self.dim_action) # initialize with zero weights
        if self.include_cash:
            self.w[-1] = 1.0  # start with all cash
        obs = self._to_obs(self.t)
        return obs, {}

    def step(self, action):
        w_target = self._project_weights(action) # get new target weights from raw actions
        turnover = np.sum(np.abs(w_target - self.w)) # calculate turnover as sum of absolute weight changes
        trading_cost = (self.commission + self.slippage) * turnover # total trading cost based on turnover

        asset_w_prev = self.w[:self.n_assets] # previous weights excluding cash
        asset_ret = np.dot(asset_w_prev, self.R[self.t]) # calculate asset return based on previous weights and next period returns
        inst_vol = np.dot(asset_w_prev, self.VOL[self.t]) # calculate instantaneous volatility based on previous weights and current volatilities

        # reward function: asset return minus trading costs and risk penalty
        reward = asset_ret - trading_cost - self.risk_lambda * inst_vol

        self.portfolio_value *= math.exp(asset_ret - trading_cost) # update portfolio value using log returns

        self.w = w_target # update calculated weights for next step
        self.t += 1 # move to next time step
        terminated = (self.t >= len(self.R)-1) # episode ends if we reach the end of the data
        truncated = False

        obs = self._to_obs(self.t) if not terminated else self._to_obs(self.t-1)
        obs = np.clip(obs, -5.0, 5.0) # clip observations to previously defined bounds (±5 for z-scored features)
        
        # Additional info for monitoring leveraged positions
        leverage = np.sum(np.abs(self.w[:self.n_assets]))
        info = {
            "portfolio_value": self.portfolio_value, 
            "turnover": turnover, 
            "inst_vol": inst_vol, 
            "asset_ret": asset_ret,
            "total_leverage": leverage
        }
        return obs, reward, terminated, truncated, info

In [ ]:
def slice_by_mask(X, R, VOL, mask: np.ndarray):
    idx = np.where(mask)[0] # get all indices where mask is True
    
    # If we have no valid indices, return empty arrays with correct shapes
    if len(idx) == 0:
        empty_shape_x = list(X.shape)
        empty_shape_x[0] = 0
        empty_shape_r = list(R.shape)
        empty_shape_r[0] = 0
        empty_shape_v = list(VOL.shape)
        empty_shape_v[0] = 0
        return np.zeros(empty_shape_x), np.zeros(empty_shape_r), np.zeros(empty_shape_v)
    
    return X[idx], R[idx], VOL[idx] # return only the selected slices of data

def make_env_from_mask(mask, name="env"):
    X_s, R_s, V_s = slice_by_mask(X_all, R_all, VOL_all, mask)
    env = PortfolioWeightsEnv(X_s, R_s, V_s, TICKER_ORDER, CONFIG["ENV"]["lookback_window"], CONFIG["ENV"])
    env = Monitor(env, filename=None)
    return env

In [ ]:
def annualize_factor(sampling: str):
    return ANNUALIZATION.get(sampling, 365*24)

def compute_metrics(equity_curve: pd.Series, sampling: str, turnover_series: pd.Series = None):
    ret = equity_curve.pct_change().dropna()
    ann = annualize_factor(sampling)
    mu = ret.mean() * ann
    sigma = ret.std() * math.sqrt(ann)
    sharpe = mu / (sigma + 1e-12)
    downside = ret[ret < 0].std() * math.sqrt(ann)
    sortino = mu / (downside + 1e-12)
    if len(equity_curve) > 1:
        # Calculate years based on number of samples and sampling frequency
        if isinstance(equity_curve.index, pd.DatetimeIndex):
            dt_years = (equity_curve.index[-1] - equity_curve.index[0]).total_seconds() / (365 * 24 * 3600)
        else:
            # If using RangeIndex, calculate based on sampling frequency
            samples = len(equity_curve)
            samples_per_year = annualize_factor(sampling)
            dt_years = samples / samples_per_year
        dt_years = float(dt_years) if float(dt_years) != 0 else 1e-12
        cagr = (equity_curve.iloc[-1] / equity_curve.iloc[0]) ** (1/dt_years) - 1
    else:
        cagr = 0.0
    cummax = equity_curve.cummax()
    dd = (equity_curve / cummax - 1).min()
    maxdd = float(dd)
    calmar = mu / (abs(maxdd) + 1e-12)
    hit_ratio = (ret > 0).mean()
    turnover = turnover_series.mean() if turnover_series is not None and len(turnover_series)>0 else np.nan
    return {"CAGR": cagr, "Sharpe": sharpe, "Sortino": sortino, "MaxDrawdown": maxdd, "Calmar": calmar, "Volatility": sigma, "Turnover": turnover, "HitRatio": hit_ratio}

def plot_series(series: pd.Series, title: str):
    plt.figure(figsize=(10,4))
    plt.plot(series.index, series.values)
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)
    plt.show()

def plot_training_periods(coint_df: pd.DataFrame, price_index: pd.DatetimeIndex):
    """Plot timeline showing training periods and their overlap."""
    plt.figure(figsize=(15,5))
    y = 0
    for _, row in coint_df.iterrows():
        plt.hlines(y, row['train_start'], row['train_end'], 'blue', alpha=0.3)
        y += 1
    plt.title("Training Periods Timeline")
    plt.xlabel("Time")
    plt.ylabel("Period Index")
    plt.grid(True)
    plt.show()

def backtest_env(env: gym.Env, model=None):
    # Get the unwrapped environment
    unwrapped = env.unwrapped if hasattr(env, 'unwrapped') else env
    
    obs, _ = env.reset()
    pv, turns = [], []
    leverage = []  # Track leverage over time
    
    for t in range(len(unwrapped.R)-1):
        if model is None:
            action = np.ones(unwrapped.dim_action)/unwrapped.dim_action
        else:
            action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, trunc, info = env.step(action)
        pv.append(info["portfolio_value"])
        turns.append(info["turnover"])
        leverage.append(info.get("total_leverage", 0))
        if done:
            break
            
    idx = pd.RangeIndex(start=0, stop=len(pv), step=1)
    ec = pd.Series(pv, index=idx)
    to = pd.Series(turns, index=idx)
    lev = pd.Series(leverage, index=idx)
    
    return ec, to, lev

# Plot cointegration training periods
print("Visualizing training periods timeline...")
plot_training_periods(coint_df, TIME_INDEX)

In [ ]:
ensure_dir(CONFIG["IO"]["models_dir"])
ensure_dir(CONFIG["EVAL"]["reports_dir"])

RESULTS = []

for split in SPLITS:
    print(f"\n=== Training on split: {split['name']} ===")
    train_env = make_env_from_mask(split["train"], name=f"{split['name']}_train")
    eval_env  = make_env_from_mask(split["test"], name=f"{split['name']}_test")

    vec_train = DummyVecEnv([lambda: train_env])
    vec_eval  = DummyVecEnv([lambda: eval_env])

    model = PPO(
        policy=CONFIG["RL"]["policy"],
        env=vec_train,
        gamma=CONFIG["RL"]["gamma"],
        gae_lambda=CONFIG["RL"]["gae_lambda"],
        clip_range=CONFIG["RL"]["clip_range"],
        n_steps=CONFIG["RL"]["n_steps"],
        batch_size=CONFIG["RL"]["batch_size"],
        learning_rate=CONFIG["RL"]["learning_rate"],
        ent_coef=CONFIG["RL"]["ent_coef"],
        vf_coef=CONFIG["RL"]["vf_coef"],
        max_grad_norm=CONFIG["RL"]["max_grad_norm"],
        tensorboard_log=CONFIG["IO"]["tb_logdir"],
        device=device,
        verbose=1
    )

    eval_callback = EvalCallback(
        vec_eval, 
        best_model_save_path=CONFIG["IO"]["models_dir"],
        log_path=CONFIG["IO"]["models_dir"], 
        eval_freq=10000,
        deterministic=True, 
        render=False
    )
    
    model.learn(total_timesteps=CONFIG["RL"]["timesteps"], callback=eval_callback)
    model_path = os.path.join(CONFIG["IO"]["models_dir"], f"ppo_{split['name']}.zip")
    model.save(model_path)
    print("Saved model:", model_path)

    test_env = make_env_from_mask(split["test"], name=f"{split['name']}_test")
    ec, to = backtest_env(test_env, model=model)

    idx = np.where(split["test"])[0]
    R_test = R_all[idx]
    ew = np.ones(len(TICKER_ORDER))/len(TICKER_ORDER)
    ec_bench = [1.0]
    # Only iterate through the same number of returns as we have in ec.index
    for i in range(len(ec.index)-1):
        ec_bench.append(ec_bench[-1]*math.exp(np.dot(ew, R_test[i])))
    ec_bench = pd.Series(ec_bench, index=ec.index)

    bh_btc, bh_eth = [1.0], [1.0]
    # Only iterate through the same number of returns as we have in ec.index
    for i in range(len(ec.index)-1):
        bh_btc.append(bh_btc[-1]*math.exp(R_test[i][0]))
        bh_eth.append(bh_eth[-1]*math.exp(R_test[i][1]))
    bh_btc = pd.Series(bh_btc, index=ec.index)
    bh_eth = pd.Series(bh_eth, index=ec.index)

    m_model = compute_metrics(ec, CONFIG["DATA"]["sampling"], to)
    m_ew    = compute_metrics(ec_bench, CONFIG["DATA"]["sampling"])
    m_btc   = compute_metrics(bh_btc, CONFIG["DATA"]["sampling"])
    m_eth   = compute_metrics(bh_eth, CONFIG["DATA"]["sampling"])

    RESULTS.append({"split": split["name"], "model": m_model, "equal_weight": m_ew, "buy_and_hold_BTC": m_btc, "buy_and_hold_ETH": m_eth})

    if CONFIG["EVAL"]["plots"]:
        plot_series(ec, f"Equity Curve — PPO ({split['name']})")
        plot_series((ec / ec.cummax()) - 1.0, f"Drawdown — PPO ({split['name']})")
        plot_series(ec_bench, f"Equity Curve — Equal-Weight Hold ({split['name']})")

print("Done. RESULTS collected.")

In [ ]:

rows = []
for res in RESULTS:
    row = {"split": res["split"]}
    for k, metrics in res.items():
        if k == "split":
            continue
        for mname, mval in metrics.items():
            row[f"{k}_{mname}"] = mval
    rows.append(row)

df_results = pd.DataFrame(rows)
df_results

In [ ]:
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
ensure_dir(CONFIG["EVAL"]["reports_dir"])
out_json = os.path.join(CONFIG["EVAL"]["reports_dir"], f"metrics_{ts}.json")
out_csv  = os.path.join(CONFIG["EVAL"]["reports_dir"], f"metrics_{ts}.csv")
df_results.to_csv(out_csv, index=False)
with open(out_json, "w") as f:
    json.dump(RESULTS, f, indent=2)
print("Saved:", out_json, "and", out_csv)